In [1]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="udacity-project")--> This can only work if I create a workspace named "udacity-project" in the Azure ML studio .
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
print(ws)

Workspace name: machinelearning
Azure region: eastus2
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-164887
Workspace.create(name='machinelearning', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-164887')


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 4, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-12-02T10:49:57.183000+00:00', 'errors': None, 'creationTime': '2021-12-02T10:44:23.905397+00:00', 'modifiedTime': '2021-12-02T10:44:39.497444+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [5]:
# from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler

ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",
            compute_target=compute_target,
            vm_size='STANDARD_D2_V2',
            entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     estimator=est,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)



'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
# Start the HyperDrive run
hyperdrive_run = exp.submit(hyperdrive_config)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
 

hyperdrive_run.wait_for_completion(show_output=True)
RunDetails(hyperdrive_run).show()

assert(hyperdrive_run.get_status() == "Completed")


RunId: HD_dc5e2a9e-fa53-4dc2-9cd9-cac44d5e6999
Web View: https://ml.azure.com/runs/HD_dc5e2a9e-fa53-4dc2-9cd9-cac44d5e6999?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-164887/workspaces/machinelearning&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-12-02T11:09:34.135669][API][INFO]Experiment created<END>\n""<START>[2021-12-02T11:09:34.724834][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-12-02T11:09:34.868894][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_dc5e2a9e-fa53-4dc2-9cd9-cac44d5e6999
Web View: https://ml.azure.com/runs/HD_dc5e2a9e-fa53-4dc2-9cd9-cac44d5e6999?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-164887/workspaces/machinelearning&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
import joblib

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

# Get your best run and save the model from that run.

# get_best_run_by_primary_metric()
# Returns the best Run, or None if no child has the primary metric.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
# get_file_names()
# Returns a list of the files that are stored in association with the run.
print("Best run file names :",best_run.get_file_names())




[{'run_id': 'HD_dc5e2a9e-fa53-4dc2-9cd9-cac44d5e6999_12', 'hyperparameters': '{"--C": 50, "--max_iter": 100}', 'best_primary_metric': 0.9165402124430956, 'status': 'Completed'}, {'run_id': 'HD_dc5e2a9e-fa53-4dc2-9cd9-cac44d5e6999_14', 'hyperparameters': '{"--C": 0.01, "--max_iter": 300}', 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_dc5e2a9e-fa53-4dc2-9cd9-cac44d5e6999_13', 'hyperparameters': '{"--C": 0.1, "--max_iter": 100}', 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_dc5e2a9e-fa53-4dc2-9cd9-cac44d5e6999_10', 'hyperparameters': '{"--C": 1000, "--max_iter": 300}', 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_dc5e2a9e-fa53-4dc2-9cd9-cac44d5e6999_8', 'hyperparameters': '{"--C": 1000, "--max_iter": 200}', 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_dc5e2a9e-fa53-4dc2-9cd9-cac44d5e6999_4', 'hyperparameters': '{"--C": 0.1, "--max_iter": 200}'

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [12]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [13]:
from azureml.train.automl import AutoMLConfig


automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    iteration_timeout_minutes=5,
    n_cross_validations=2)










In [21]:
# Submit your automl run
#from azureml.core.experiment import Experiment


#remote_run = exp.submit(automl_config, show_output = False)
from azureml.core.experiment import Experiment

experiment = Experiment(ws, "automl_test_experiment")
run = experiment.submit(config=automl_config, show_output=True)


Submitting remote run.
Running on remote compute: compute-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl_test_experiment,AutoML_a56e8b9b-756c-4d68-807c-8d43147a2258,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

In [23]:
# Retrieve and save your best automl model.
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.35.1, current version:1.34.0
Package:azureml-core, training version:1.35.0.post1, current version:1.34.0
Package:azureml-dataprep, training version:2.23.2, current version:2.22.2
Package:azureml-dataprep-rslex, training version:1.21.2, current version:1.20.1
Package:azureml-dataset-runtime, training version:1.35.0, current version:1.34.0
Package:azureml-defaults, training version:1.35.0, current version:1.34.0
Package:azureml-interpret, training version:1.35.0, current version:1.34.0
Package:azureml-mlflow, training version:1.35.0, current version:1.34.0
Package:azureml-pipeline-core, training version:1.35.0, current version:1.34.0
Package:azureml-responsibleai, training version:1.35.0, current version:1.34.0
Package:azureml-telemetry, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-client, training version:1.35.0, current version:1.34.0
Package:azureml-train-automl-runtime, training version:1.35.1, current

Run(Experiment: automl_test_experiment,
Id: AutoML_a56e8b9b-756c-4d68-807c-8d43147a2258_11,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=None))], verbose=False)), ('7', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=1.7575106248547894, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('2', Pipeline(memory=None, steps=[('max

In [24]:
best_run.get_metrics()

{'norm_macro_recall': 0.47640250020287644,
 'log_loss': 0.21347504056161207,
 'average_precision_score_micro': 0.981135104268435,
 'AUC_micro': 0.9804655197901819,
 'f1_score_macro': 0.7644418216052982,
 'average_precision_score_weighted': 0.9549769380693949,
 'recall_score_weighted': 0.9159028831562974,
 'precision_score_macro': 0.801405392719245,
 'AUC_macro': 0.9464806591378747,
 'f1_score_weighted': 0.9109972573531921,
 'matthews_correlation': 0.5357248625185906,
 'precision_score_micro': 0.9159028831562974,
 'recall_score_macro': 0.7382012501014382,
 'AUC_weighted': 0.9464806591378747,
 'precision_score_weighted': 0.908808153008863,
 'average_precision_score_macro': 0.8238269034873813,
 'balanced_accuracy': 0.7382012501014382,
 'accuracy': 0.9159028831562974,
 'weighted_accuracy': 0.9600439006800229,
 'f1_score_micro': 0.9159028831562974,
 'recall_score_micro': 0.9159028831562974,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_a56e8b9b-756c-4d68-807c-8d43147a2258_1

In [25]:
fitted_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
), random_state=None))], verbose=False)), ('7', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=1.7575106248547894, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))], verbose=False)), ('2', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('extratreesclassifier', ExtraTre

In [26]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl_test_experiment,AutoML_a56e8b9b-756c-4d68-807c-8d43147a2258_11,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
# Cluster clean up

compute_target.delete()